<a href="https://colab.research.google.com/github/Apple03244/DACON/blob/main/%EA%B3%A0%EA%B0%9D_%EB%8C%80%EC%B6%9C%EB%93%B1%EA%B8%89_%EB%B6%84%EB%A5%98_%ED%95%B4%EC%BB%A4%ED%86%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 고객 대출등급 분류 해커톤

In [1]:
import pandas as pd

In [2]:
train=pd.read_csv('/Users/ijeonghun/Documents/projects/dacon/고객 대출등급 분류 해커톤/train.csv')

In [3]:
train.head(3)

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,TRAIN_00000,12480000,36 months,6 years,RENT,72000000,18.90,15,부채 통합,0,0,0.0,0.0,0.0,C
1,TRAIN_00001,14400000,60 months,10+ years,MORTGAGE,130800000,22.33,21,주택 개선,0,373572,234060.0,0.0,0.0,B
2,TRAIN_00002,12000000,36 months,5 years,MORTGAGE,96000000,8.60,14,부채 통합,0,928644,151944.0,0.0,0.0,A


In [4]:
import sklearn.preprocessing as skpre
import sklearn.model_selection as skmod
import sklearn.pipeline as skpip
import sklearn.base as skbase
import sklearn.compose as skcom

In [5]:
import matplotlib.pyplot as plt
import seaborn as sb

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96294 entries, 0 to 96293
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            96294 non-null  object 
 1   대출금액          96294 non-null  int64  
 2   대출기간          96294 non-null  object 
 3   근로기간          96294 non-null  object 
 4   주택소유상태        96294 non-null  object 
 5   연간소득          96294 non-null  int64  
 6   부채_대비_소득_비율   96294 non-null  float64
 7   총계좌수          96294 non-null  int64  
 8   대출목적          96294 non-null  object 
 9   최근_2년간_연체_횟수  96294 non-null  int64  
 10  총상환원금         96294 non-null  int64  
 11  총상환이자         96294 non-null  float64
 12  총연체금액         96294 non-null  float64
 13  연체계좌수         96294 non-null  float64
 14  대출등급          96294 non-null  object 
dtypes: float64(4), int64(5), object(6)
memory usage: 11.0+ MB


In [7]:
train["대출기간"].value_counts()

대출기간
 36 months    64479
 60 months    31815
Name: count, dtype: int64

In [8]:
train["근로기간"].value_counts()

근로기간
10+ years    31585
2 years       8450
< 1 year      7774
3 years       7581
1 year        6249
Unknown       5671
5 years       5665
4 years       5588
8 years       4888
6 years       3874
7 years       3814
9 years       3744
10+years       896
<1 year        370
3               89
1 years         56
Name: count, dtype: int64

In [9]:
train["대출목적"].value_counts()

대출목적
부채 통합     55150
신용 카드     24500
주택 개선      6160
기타         4725
주요 구매      1803
의료         1039
자동차         797
소규모 사업      787
이사          506
휴가          466
주택          301
재생 에너지       60
Name: count, dtype: int64

In [10]:
train["주택소유상태"].value_counts()

주택소유상태
MORTGAGE    47934
RENT        37705
OWN         10654
ANY             1
Name: count, dtype: int64

In [11]:
train["최근_2년간_연체_횟수"].value_counts()

최근_2년간_연체_횟수
0     76392
1     13079
2      3896
3      1430
4       697
5       338
6       178
7        86
8        67
9        40
10       35
11       21
12        9
14        8
13        8
16        4
18        2
15        2
17        1
30        1
Name: count, dtype: int64

### 1. 전처리

In [17]:
import numpy as np
import sklearn.base as skbase
import sklearn.impute as skimp
import sklearn.compose as skcom
import sklearn.preprocessing as skpre
import sklearn.pipeline as skpip



#근로기간 전처리 -> ordinal
class make_ord(skbase.TransformerMixin,skbase.BaseEstimator):
  def __init__(self) -> None:
    super().__init__()
  def fit(self,X,y=None):
    self.columns_names=list(X.columns)
    return self
  def get_feature_names_out(self):
    return self.columns_names

  def transform(self,X,y=None):
    def trans(data):
      if data=="Unknown":
        return np.nan

      # 여기가 까다로운 부분
      if type(data)==str:
        condi_1="<" in data
        condi_2="1" in data
        condi_3="year" in data
        if all([condi_1,condi_2,condi_3]):
          return 0

        condi_4="+" in data
        condi_5="10" in data
        if all([condi_4,condi_5]):
          return 10
        else:
          result=[]
          for letter in data:
            if letter.isnumeric():
              result.append(letter)
          return int("".join(result))
    return X.applymap(trans)
  
class passthrough(skbase.BaseEstimator,skbase.TransformerMixin):
  def __init__(self) -> None:
    super().__init__()
  def fit(self,X,y=None):
    self.columns=X.columns
  def get_feature_names_out(self):
    return self.columns
  def transform(self,X,y=None):
    return X
  
class preprocess(skbase.BaseEstimator,skbase.TransformerMixin):
  def __init__(self) -> None:
    super().__init__()
  def fit(self,X,y=None):
    self.numeric_columns=X.select_dtypes(np.number).columns
    self.cate_columns=X.select_dtypes("object").columns.difference(["대출등급",'ID',"근로기간"])
    self.target_column="대출등급"
    
  

In [18]:
test=make_ord()

In [19]:
test.fit(train[["근로기간"]])

make_ord()

/var/folders/bv/d7dzvp616v1d6dz4vbkn7zw40000gn/T/ipykernel_14761/2787672213.py:43: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return X.applymap(trans)


근로기간
10.0    32481
2.0      8450
0.0      8144
3.0      7670
1.0      6305
5.0      5665
4.0      5588
8.0      4888
6.0      3874
7.0      3814
9.0      3744
Name: count, dtype: int64

In [ ]:
numeric_columns=train.select_dtypes(np.number).columns
cate_columns=train.select_dtypes("object").columns.difference(["대출등급",'ID',"근로기간"])
target_column="대출등급"

In [ ]:
plt.figure()
sb.pairplot()

TypeError: pairplot() missing 1 required positional argument: 'data'

<Figure size 640x480 with 0 Axes>

In [ ]:
import autoviz

ModuleNotFoundError: No module named 'autoviz'